**This notebook is an exercise in the [SQL](https://www.kaggle.com/learn/intro-to-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/dansbecker/joining-data).**

---


# Introduction

[Stack Overflow](https://stackoverflow.com/) is a widely beloved question and answer site for technical questions. You'll probably use it yourself as you keep using SQL (or any programming language). 

Their data is publicly available. What cool things do you think it would be useful for?

Here's one idea:
You could set up a service that identifies the Stack Overflow users who have demonstrated expertise with a specific technology by answering related questions about it, so someone could hire those experts for in-depth help.

In this exercise, you'll write the SQL queries that might serve as the foundation for this type of service.

As usual, run the following cell to set up our feedback system before moving on.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex6 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


Run the next cell to fetch the `stackoverflow` dataset.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

### 1) Explore the data

Before writing queries or **JOIN** clauses, you'll want to see what tables are available. 

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [12]:
# Get a list of available tables 
tables = list(client.list_tables(dataset)) # Your code here
list_of_tables = [table.table_id for table in tables]

# Print your answer
print(list_of_tables)

# Check your answer
q_1.check()

['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [13]:
#q_1.solution()

### 2) Review relevant tables

If you are interested in people who answer questions on a given topic, the `posts_answers` table is a natural place to look. Run the following cell, and look at the output.

In [14]:
# Construct a reference to the "posts_answers" table
answers_table_ref = dataset_ref.table("posts_answers")

# API request - fetch the table
answers_table = client.get_table(answers_table_ref)

# Preview the first five lines of the "posts_answers" table
client.list_rows(answers_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,64548693,None,<p>My workaround without ejecting:</p>\n<ol>\n...,None,None,0,None,2020-10-27 05:20:53.417000+00:00,None,2020-10-27 05:20:53.417000+00:00,None,None,None,None,9428719,55821078,2,0,None,None
1,64548694,None,<p>The execution thread may be secluded on dif...,None,None,1,None,2020-10-27 05:21:15.397000+00:00,None,2020-10-27 05:21:15.397000+00:00,None,None,None,None,13927193,33876455,2,0,None,None
2,64548698,None,"<p><code>vw</code> is well supported, so can b...",None,None,0,None,2020-10-27 05:22:37.873000+00:00,None,2020-10-27 05:22:37.873000+00:00,None,None,None,None,8942566,64548101,2,0,None,None
3,64548705,None,<p>This could be simple. Please check that you...,None,None,0,None,2020-10-27 05:23:43.193000+00:00,None,2020-10-27 05:23:43.193000+00:00,None,None,None,None,13832463,64276190,2,0,None,None
4,64548730,None,<p>Install the flutter plugin on android studi...,None,None,3,None,2020-10-27 05:26:53.640000+00:00,None,2020-10-27 05:26:53.640000+00:00,None,None,None,None,11211493,64443398,2,0,None,None


It isn't clear yet how to find users who answered questions on any given topic. But `posts_answers` has a `parent_id` column. If you are familiar with the Stack Overflow site, you might figure out that the `parent_id` is the question each post is answering.

Look at `posts_questions` using the cell below.

In [15]:
# Construct a reference to the "posts_questions" table
questions_table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
questions_table = client.get_table(questions_table_ref)

# Preview the first five lines of the "posts_questions" table
client.list_rows(questions_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,27270264,"Javascript, strange recursion behavior, DFS",<p>So I had a simple task to implement DFS in ...,27270865.0,1,0,None,2014-12-03 11:13:04.137000+00:00,NaN,2017-01-27 19:06:57.580000+00:00,2017-01-27 19:06:57.580000+00:00,None,3768871.0,None,1377178,None,1,0,javascript|recursion|tree|depth-first-search,256
1,27273092,Generating Windup report for a .java file (JBo...,<p>I have gone through few demos on generating...,NaN,2,0,None,2014-12-03 13:35:36.393000+00:00,NaN,2017-07-10 15:32:05.833000+00:00,2017-07-10 15:32:05.833000+00:00,None,1432.0,None,3972814,None,1,1,java|jboss|migration|rhamt,256
2,27284272,Execute multiple %s commands in VIM always giv...,<p>I know that we have many questions (and ans...,27288235.0,2,0,None,2014-12-04 00:33:34.713000+00:00,NaN,2014-12-04 14:23:18.773000+00:00,2014-12-04 01:31:54.673000+00:00,None,1470607.0,None,179589,None,1,1,vim,256
3,27285068,Offset value in Image data matrix and its effe...,<p>Currently I am trying to understand the Ope...,27288516.0,1,0,None,2014-12-04 02:07:11.620000+00:00,1.0,2014-12-06 05:34:53.850000+00:00,2014-12-04 02:14:28.137000+00:00,None,1470607.0,None,2467772,None,1,0,c++|c|opencv|matrix,256
4,27290664,Use of unresolved identifier vDSP_vsq in Xcode...,<p>I'm using Accelerate.framework to get some ...,27307977.0,1,0,None,2014-12-04 09:40:52.767000+00:00,NaN,2014-12-05 02:53:23.950000+00:00,NaT,None,NaN,None,1822879,None,1,0,ios|xcode|accelerate-framework,256


Are there any fields that identify what topic or technology each question is about? If so, how could you find the IDs of users who answered questions about a specific topic?

Think about it, and then check the solution by running the code in the next cell.

In [16]:
# Check your answer (Run this code cell to receive credit!)
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
`posts_questions` has a column called `tags` which lists the topics/technologies each question is about.

`posts_answers` has a column called `parent_id` which identifies the ID of the question each answer is responding to.
`posts_answers` also has an `owner_user_id` column which specifies the ID of the user who answered the question.

You can join these two tables to:
- determine the `tags` for each answer, and then
- select the `owner_user_id` of the answers on the desired tag.

This is exactly what you will do over the next few questions.


### 3) Selecting the right questions

A lot of this data is text. 

We'll explore one last technique in this course which you can apply to this text.

A **WHERE** clause can limit your results to rows with certain text using the **LIKE** feature. For example, to select just the third row of the `pets` table from the tutorial, we could use the query in the picture below.

![](https://i.imgur.com/RccsXBr.png) 

You can also use `%` as a "wildcard" for any number of characters. So you can also get the third row with:

```
query = """
        SELECT * 
        FROM `bigquery-public-data.pet_records.pets` 
        WHERE Name LIKE '%ipl%'
        """
```

Try this yourself. Write a query that selects the `id`, `title` and `owner_user_id` columns from the `posts_questions` table. 
- Restrict the results to rows that contain the word "bigquery" in the `tags` column. 
- Include rows where there is other text in addition to the word "bigquery" (e.g., if a row has a tag "bigquery-sql", your results should include that too).

In [21]:
# Your code here
questions_query = """
                  SELECT id, title, owner_user_id
                  FROM `bigquery-public-data.stackoverflow.posts_questions`
                  WHERE tags LIKE '%bigquery%'
                  """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
questions_query_job = client.query(questions_query) # Your code goes here

# API request - run the query, and return a pandas DataFrame
questions_results = questions_query_job.to_dataframe() # Your code goes here

# Preview results
print(questions_results.head())

# Check your answer
q_3.check()

         id                                              title  owner_user_id
0  18116803  Getting 401 uploading file into a table with a...       508960.0
1  18050595  Workaround needed for the incomplete implement...      2264178.0
2  17997089      BigQuery rank() function - Resources exceeded      2276668.0
3  17868028         Loop through all BigQuery jobs with Python      2436853.0
4  17885149  BigQuery. Error when querying multiple tables:...      2623133.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [22]:
#q_3.hint()
#q_3.solution()

### 4) Your first join
Now that you have a query to select questions on any given topic (in this case, you chose "bigquery"), you can find the answers to those questions with a **JOIN**.  

Write a query that returns the `id`, `body` and `owner_user_id` columns from the `posts_answers` table for answers to "bigquery"-related questions. 
- You should have one row in your results for each answer to a question that has "bigquery" in the tags.  
- Remember you can get the tags for a question from the `tags` column in the `posts_questions` table.

Here's a reminder of what a **JOIN** looked like in the tutorial:
```
query = """
        SELECT p.Name AS Pet_Name, o.Name AS Owner_Name
        FROM `bigquery-public-data.pet_records.pets` as p
        INNER JOIN `bigquery-public-data.pet_records.owners` as o 
            ON p.ID = o.Pet_ID
        """
```

It may be useful to scroll up and review the first several rows of the `posts_answers` and `posts_questions` tables.  

In [28]:
# Your code here
answers_query = """
                SELECT a.id, a.body, a.owner_user_id
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q 
                INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                    ON q.id = a.parent_id
                WHERE q.tags LIKE '%bigquery%'
                """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
answers_query_job = client.query(answers_query) # Your code goes here

# API request - run the query, and return a pandas DataFrame
answers_results = answers_query_job.to_dataframe() # Your code goes here

# Preview results
print(answers_results.head())

# Check your answer
q_4.check()

         id                                               body  owner_user_id
0  11836963  <p>I've found exporting from a big query table...      1579131.0
1  11957891  <p>There should be an additional newline after...      1362175.0
2  11967957  <p>Seems that it is possible to download the r...      1165239.0
3  12012982  <p>Based on what I can observe from outside go...       563868.0
4  11562597  <p>It turned out to be a problem with the open...       191790.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [29]:
#q_4.hint()
#q_4.solution()

### 5) Answer the question
You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that has a single row for each user who answered at least one question with a tag that includes the string "bigquery". Your results should have two columns:
- `user_id` - contains the `owner_user_id` column from the `posts_answers` table
- `number_of_answers` - contains the number of answers the user has written to "bigquery"-related questions

In [36]:
# Your code here
bigquery_experts_query = """
                        SELECT pa.owner_user_id AS user_id, COUNT(1) AS number_of_answers
                        FROM `bigquery-public-data.stackoverflow.posts_questions` AS pq
                        INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS pa
                        ON pq.id = pa.parent_Id
                        WHERE pq.tags LIKE "%bigquery%"
                        GROUP BY pa.owner_user_id
                        """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query) # Your code goes here

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe() # Your code goes here

# Preview results
print(bigquery_experts_results.head())

# Check your answer
q_5.check()

      user_id  number_of_answers
0   8146942.0                  1
1  14048579.0                 84
2     19880.0                 62
3   5679311.0                  1
4   6345474.0                  1


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [30]:
#q_5.hint()
#q_5.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

bigquery_experts_query = """
                         SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
                         FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                         INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                             ON q.id = a.parent_Id
                         WHERE q.tags LIKE '%bigquery%'
                         GROUP BY a.owner_user_id
                         """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe()

```

### 6) Building a more generally useful service

How could you convert what you've done to a general function a website could call on the backend to get experts on any topic?  

Think about it and then check the solution below.

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_6.solution()

# Congratulations!

You know all the key components to use BigQuery and SQL effectively. Your SQL skills are sufficient to unlock many of the world's largest datasets.

Want to go play with your new powers?  Kaggle has BigQuery datasets available [here](https://www.kaggle.com/datasets?fileType=bigQuery).

---




*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161314) to chat with other Learners.*